### Entropy metric
<img src="img/aic.png">

#### Dependencies
```sh
pip install liac-arff
```

In [1]:
import math

In [2]:
class EntropyMetric(object):
    """Entropy metric H(Bs, D) module (Bs - Bayes net, D - test data set)"""
    def __init__(self):
        # Liczba probek testowych w D
        self.N = 3
        # self.N = len(D)
        # Testowa siec Bs, X0 -> X1
        self.Bs = [{'r': 3, 'q': 1, 'name': 'X0', 'parent': ''}, {'r': 3, 'q': 3, 'name': 'X1', 'parent': 'X0'}] 
        # Dane testowe
        self.D = [ {'X0': 1, 'X1': 1}, {'X0': 1, 'X1': 1}, {'X0': 2, 'X1': 1} ]
        
    '''node_state = k, parent_state = j, node index = i'''
    def check_occurences(self, node_name, parent_name, node_state, parent_state):
        #print("FOR ", node_name, "=", node_state, " ", parent_name, "=", parent_state)
        result = 0
        # brak parenta
        if (parent_name == ''):
            return result

        for i, p in enumerate(self.D):
            found_node_state = p[node_name]
            found_parent_state = p[parent_name]
            is_found = (found_node_state == node_state and found_parent_state == parent_state)
            #print(p, is_found)
            if (is_found):
                result += 1

        return result

    '''parent_state = j'''
    def check_parent_occurences(self, parent_name, parent_state):
        #print("FOR parent ", parent_name, "=", parent_state)
        result = 0
        # brak parenta
        if (parent_name == ''):
            return result

        for i, p in enumerate(self.D):
            if (p[parent_name] == parent_state):
                #print("found!")
                result += 1

        return result
    
    def compute_entropy_metric(self):
        # Pierwsza iteracja (póki co ręcznie)
        result = 0

        for i in range(0, len(self.Bs)):
            r_i = self.Bs[i - 1]['r'] #r(i), czyli tak naprawdę r(i - 1)
            q_i = self.Bs[i - 1]['q'] #q(i), czyli tak naprawdę q(i - 1)
            
            for j in range(1, q_i + 1):
                for k in range(1, r_i + 1):
                    N_i_j_k = self.check_occurences(self.Bs[i]['name'], self.Bs[i]['parent'], k, j)
                    N_i_j = self.check_parent_occurences(self.Bs[i]['parent'], j)

                    if (N_i_j_k != 0 and N_i_j != 0):
                        result += (N_i_j_k / self.N) * math.log10(N_i_j_k / N_i_j)
                        
        return -1 * self.N * result

------

### K2 algorithm

In [ ]:
class K2Algorithm(object):
    def __init__(self, D, scoring_method):
        # D - test data
        self.D = D
        self.scoring_method = scoring_method
        
    def find_optimal_net(self):
        return "found"

------

### ARFF file loading

In [13]:
import arff
import pprint

pp = pprint.PrettyPrinter(depth=6)

with open('data/weather.arff') as fh:
    data = arff.load(fh)
    
    attributes = []
    for i, p in enumerate(data['attributes']):
        attributes.append({'name': p[0], 'states': p[1]})
    
    sample_data = []
    for i, p in enumerate(data['data']):
        temp_dict = []
        for j, d in enumerate(p):
            temp_dict.append({attributes[j]['name']: d})
        sample_data.append(temp_dict)
        
    index_of_class_attribute = 4
    print("Class argument: ")
    pp.pprint(attributes[index_of_class_attribute])

Class argument: 
{'name': 'play', 'states': ['yes', 'no']}


In [14]:
    final_attributes = []
    final_attributes.append(attributes[index_of_class_attribute])
    
    for i, p in enumerate(attributes):
        if i != index_of_class_attribute:
            final_attributes.append(p)
    
    pp.pprint(final_attributes)
    # self.Bs = [{'r': 3, 'q': 1, 'name': 'play', 'parent': ''}, {'r': 3, 'q': 3, 'name': 'X1', 'parent': 'X0'}] 

[{'name': 'play', 'states': ['yes', 'no']},
 {'name': 'outlook', 'states': ['sunny', 'overcast', 'rainy']},
 {'name': 'temperature', 'states': ['hot', 'mild', 'cool']},
 {'name': 'humidity', 'states': ['high', 'normal']},
 {'name': 'windy', 'states': ['TRUE', 'FALSE']}]


-----
### Main algorithm